In [1]:
import sys, os
import boto3

import pandas as pd

In [2]:
in_file_list = '/home/wb411133/temp/World_Bank_Global_3_Complete.csv'
out_folder = os.path.join(os.path.dirname(in_file_list), "FATHOM")
s3_bucket = 'wbg-geography01'
s3_prefix = 'FATHOM/v2023/'
s3_out = os.path.join('s3://',s3_bucket, s3_prefix)

in_files = pd.read_csv(in_file_list)
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(s3_bucket)

In [3]:
in_files.head()

,Layer,AWS_Path
0,GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDE...,s3://fathom-products-global/fathom-global/v3/G...
1,GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-UNDEFEN...,s3://fathom-products-global/fathom-global/v3/G...
2,GLOBAL-1ARCSEC-NW_OFFSET-1in10-FLUVIAL-DEFENDE...,s3://fathom-products-global/fathom-global/v3/G...
3,GLOBAL-1ARCSEC-NW_OFFSET-1in10-FLUVIAL-UNDEFEN...,s3://fathom-products-global/fathom-global/v3/G...
4,GLOBAL-1ARCSEC-NW_OFFSET-1in10-PLUVIAL-DEFENDE...,s3://fathom-products-global/fathom-global/v3/G...


In [4]:
# Find all files already copied
all_folders = []
for obj in my_bucket.objects.filter(Prefix=s3_prefix):
    all_folders.append(obj.key.split("/")[-2])


In [5]:
processed_folders = list(set(all_folders))
delivered_folders = in_files['Layer'].values
sel_folders = [x for x in delivered_folders if not x in processed_folders]
sel_files = in_files.loc[in_files['Layer'].isin(sel_folders)].copy()
sel_files

,Layer,AWS_Path


In [36]:
with open(os.path.join(out_folder, "aaa_download_upload_2.sh"), 'w') as out_file:
    out_file.write('#!/bin/bash\n')
    for idx, row in sel_files.iterrows():
        fathom_path = row['AWS_Path']
        local_folder = os.path.join(out_folder, row['Layer'])
        gost_folder = os.path.join(s3_out, row['Layer'])
        if not os.path.exists(local_folder):
            os.makedirs(local_folder)
        
        cur_out_folder = os.path.join(s3_prefix, row['Layer'])
        obj_count = 0
        for obj in my_bucket.objects.filter(Prefix=cur_out_folder):
            obj_count += 1
        print(f"{row['Layer']}: {obj_count}")
        if obj_count == 0:
            download_command = f'aws s3 sync --profile fathom {fathom_path} {local_folder}'
            upload_command = f'aws s3 sync {local_folder} {gost_folder}'
            remove_command = f'rm -R {local_folder}'

            out_file.write(download_command)
            out_file.write('\n')
            out_file.write(upload_command)
            out_file.write('\n')
            out_file.write(remove_command)
            out_file.write('\n')

GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 2567
GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-UNDEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in10-FLUVIAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in10-FLUVIAL-UNDEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in10-PLUVIAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in100-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in100-COASTAL-UNDEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in100-FLUVIAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in100-FLUVIAL-UNDEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in100-PLUVIAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in1000-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0: 0
GLO

GLOBAL-1ARCSEC-NW_OFFSET-1in20-FLUVIAL-DEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in20-FLUVIAL-UNDEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in20-PLUVIAL-DEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in200-COASTAL-DEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in200-COASTAL-UNDEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in200-FLUVIAL-DEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in200-FLUVIAL-UNDEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in200-PLUVIAL-DEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in5-COASTAL-DEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in5-COASTAL-UNDEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in5-FLUVIAL-DEFENDED-DEPTH-2080-SSP1_2.6-PERCENTILE50-v3.0: 0
GLOBAL-1ARC

GLOBAL-1ARCSEC-NW_OFFSET-1in5-PLUVIAL-DEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in50-COASTAL-DEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in50-COASTAL-UNDEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in50-FLUVIAL-DEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in50-FLUVIAL-UNDEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in50-PLUVIAL-DEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in500-COASTAL-DEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in500-COASTAL-UNDEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in500-FLUVIAL-DEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in500-FLUVIAL-UNDEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2080-SSP3_7.0-PERCENTILE50-v3.0: 0
GLOBAL-1A

In [20]:
all_vals = []
for idx, row in in_files.iterrows():
    all_vals.append(row['Layer'].split('-'))

In [39]:
xx = pd.DataFrame(all_vals, columns=['GLOBAL', "Size",'Offset','return','type','defense','depth','year','projection','v1','v2'])
xx.head()


,GLOBAL,Size,Offset,return,type,defense,depth,year,projection,v1,v2
0,GLOBAL,1ARCSEC,NW_OFFSET,1in1000,COASTAL,DEFENDED,DEPTH,2020,PERCENTILE50,v3.0/,None
1,GLOBAL,1ARCSEC,NW_OFFSET,1in1000,COASTAL,UNDEFENDED,DEPTH,2020,PERCENTILE50,v3.0/,None
2,GLOBAL,1ARCSEC,NW_OFFSET,1in1000,FLUVIAL,DEFENDED,DEPTH,2020,PERCENTILE50,v3.0/,None
3,GLOBAL,1ARCSEC,NW_OFFSET,1in1000,FLUVIAL,UNDEFENDED,DEPTH,2020,PERCENTILE50,v3.0/,None
4,GLOBAL,1ARCSEC,NW_OFFSET,1in1000,PLUVIAL,DEFENDED,DEPTH,2020,PERCENTILE50,v3.0/,None


In [40]:
xx['projection'].value_counts()

SSP1_2.6        80
SSP2_4.5        80
SSP5_8.5        80
PERCENTILE50    40
Name: projection, dtype: int64

In [44]:
xx.loc[xx['projection'] == 'SSP1_2.6']['year'].value_counts()

2030    40
2050    40
Name: year, dtype: int64

In [19]:
xx = rasterio.open('https://data.worldpop.org/GIS/Population/Global_2000_2020_Constrained/2020/BSGM/DZA/dza_ppp_2020_constrained.tif')

RasterioIOError: Range downloading not supported by this server!

In [37]:
5008642/1024/1024

4.776613235473633